In [25]:
from copy import deepcopy

In [26]:
class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = deepcopy(adjacency_list)
    
    def __str__(self):
        return str(self.adjacency_list)
    
    def get_all_nodes(self):
        return list(self.adjacency_list.keys())
    
    def get_neighbors(self, node):
        return self.adjacency_list[node]
    
    def add_neighbor(self, node, neighbor):
        self.adjacency_list[node].append(neighbor)
    
    def remove_neighbor(self, node, neighbor):
        self.adjacency_list[node].remove(neighbor)
    
    def add_node(self, node):
        if node not in self.get_all_nodes():
            self.adjacency_list[node] = []
           
    def outbound_edges(self, node):
        neighbors = self.get_neighbors(node)
        edges = [(node, neighbor) for neighbor in neighbors]
        return edges
    
    def inbound_edges(self, node):
        edges = []
        
        for key, neighbors in self.adjacency_list.items():
            for neighbor in neighbors:
                if neighbor == node:
                    edges.append((key, node))
                    
        return edges    
        
    # izlazni stepen cvora
    def out_degree(self, node):
        return len(self.outbound_edges(node))
    
    # ulazni stepen cvora
    def in_degree(self, node):
        return len(self.inbound_edges(node))  
    
    def get_all_edges(self):
        edges = []
        
        for node in self.get_all_nodes():
            edges += self.outbound_edges(node)
            
        return edges    
    
    def get_unvisited_edges(self, node, unvisited_edges):
        node_edges = self.outbound_edges(node)
        
        node_unvisited_edges = []
        
        for egde in unvisited_edges:
            (node_out, node_in) = egde
            if node_out == node:
                node_unvisited_edges.append(egde)
                
        return node_unvisited_edges        
    
    def has_unvisited_edges(self, node, unvisited_edges):
        unvisited_edges = self.get_unvisited_edges(node, unvisited_edges)
        
        return len(unvisited_edges) > 0
    
    def eulerian_cycle(self):
        cycle = []
        unvisited_edges = self.get_all_edges()
        current_node = self.get_all_nodes()[0]
        
        while len(unvisited_edges) > 0:
            current_cycle = []
            
            while self.has_unvisited_edges(current_node, unvisited_edges):
                unvisited_edges_from_current_node = self.get_unvisited_edges(current_node, unvisited_edges)
                selected_edge = unvisited_edges_from_current_node[0]
                unvisited_edges.remove(selected_edge)
                
                (node_out, node_in) = selected_edge
                current_cycle.append(node_in)
                current_node = node_in            
                
            cycle += current_cycle
            
            for i in range(len(cycle)):
                node = cycle[i]
                
                if self.has_unvisited_edges(node, unvisited_edges):
                    cycle = cycle[i+1:] + cycle[i+1]
                    current_node = node
                    break
                    
        return [cycle[-1]] + cycle
    
    # formiranje obilaznog puta
    # radimo prevezivanje u grafu za cvorove 'u', 'v' i 'w'
    # dodajuci novi cvor 'v*' i obilazni put preko njega
    def bypass(self, u, v, w):
        new_node = v + '*' * self.in_degree(v)
        self.add_node(new_node)
        
        self.remove_neighbor(u, v)
        self.remove_neighbor(v, w)
        self.add_neighbor(u, new_node)
        self.add_neighbor(new_node, w)
    
    # da li je graf prost (svaki cvor ima ulazni stepen <= 1)
    def is_simple(self):
        for node in self.get_all_nodes():
            if self.in_degree(node) > 1:
                return False
            
        return True
    
    def get_unvisited_neighbors(self, node, visited_nodes):
        unvisited_neighbors = []
        
        for neighbor in self.get_neighbors(node):
                if neighbor not in visited_nodes:
                    unvisited_neighbors.append(neighbor)
                    
        return unvisited_neighbors         
    
    def has_unvisited_neighbors(self, node, visited_nodes):
        unvisited_neighbors = self.get_unvisited_neighbors(node, visited_nodes)
        
        return len(unvisited_neighbors) > 0        
    
    # da li je graf povezan - DFS
    def is_connected(self):
        visited_nodes = set([])                                
        begin_node = self.get_all_nodes()[0] # pocetak je proizvoljan
        stack = [begin_node]
        
        while len(stack) > 0:
            current_node = stack[-1]
            visited_nodes.add(current_node)
            
            if self.has_unvisited_neighbors(current_node, visited_nodes):
                unvisited_neighbors = self.get_unvisited_neighbors(current_node, visited_nodes)
                stack.append(unvisited_neighbors[0]) # dodajemo samo jednog (proizvoljnog) suseda, ne sve
            else:
                stack.pop() 
                
        total_num_nodes = len(self.get_all_nodes())
        
        return len(visited_nodes) == total_num_nodes            
    
    def all_eulerian_cycles(self):
        simple_graphs = []
        non_simple_graphs = []
        
        if self.is_simple():
            simple_graphs.append(self)    
        else:
            non_simple_graphs.append(self)
        
        # svaki non-simple graf uproscavamo dok ne postane simple pravljenjem bypass-ova    
        while len (non_simple_graphs) > 0:
            current_graph = non_simple_graphs[0]
            
            # pronalazimo (proizvoljni) non-simple cvor (koji ima ulazni a samim tim i izlazni stepen >1)
            # takav cvor mora da postoji s obzirom da graf nije prost! 
            non_simple_node = None
            for node in current_graph.get_all_nodes():
                if current_graph.in_degree(node) > 1:
                    non_simple_node = node
                    break
                    
            # pravimo novi graf sa bypass-om na tom cvoru tj. odjednom pravimo za sve moguce bypass-ove
            # (za sve moguce kombinacije parove grana u/iz) na jednom cvoru po jedan novi (zaseban) graf
            non_simple_node_inbound_edges = current_graph.inbound_edges(non_simple_node)
            non_simple_node_outbound_edges = current_graph.outbound_edges(non_simple_node)
            
            for (u, _) in non_simple_node_inbound_edges:
                for (_, w) in non_simple_node_outbound_edges:
                    if u == non_simple_node and non_simple_node == w: # ciklus ne raspetljavamo pomocu bypass-a
                        continue                                     
                    
                    new_graph = Graph(current_graph.adjacency_list)
                    new_graph.bypass(u, non_simple_node, w)
                    
                    if new_graph.is_connected():
                        if new_graph.is_simple():
                            simple_graphs.append(new_graph)
                        else:
                            non_simple_graphs.append(new_graph)
            
            non_simple_graphs.remove(current_graph)
           
        all_cycles = []    
        for graph in simple_graphs:
            cycle = graph.eulerian_cycle()
            all_cycles.append([node.replace('*', '') for node in cycle])
            
        deduplicated_cycles = set([tuple(cycle) for cycle in all_cycles])    
            
        return deduplicated_cycles    
    
    # metod koji pronalazi cvorove ciji se ulazni stepen razlikuje od izlaznog
    def get_unbalanced_nodes(self):
        unbalanced_nodes = []
        
        for node in self.get_all_nodes():
            if self.in_degree(node) != self.out_degree(node):
                unbalanced_nodes.append(node)
                
        return unbalanced_nodes
    
    # metod koji povezuje nebalansirane cvorove (to su pocetni i krajnji cvor kod De Brujin-ovog grafa) 
    # kako bi osigurali uslov da graf sadrzi Ojlerov ciklus (ulazni stepen == izlazni stepen, za svaki cvor)
    def close_to_cycle(self):
        [u, v] = self.get_unbalanced_nodes()
        
        if self.in_degree(u) > self.out_degree(u):
            self.add_neighbor(u, v)
        else:
            self.add_neighbor(v, u)

In [27]:
class DeBruijn(Graph):    
    # Sjecenje DNK niske na k-grame  
    def get_kmers(self, reads, k):
        kmers = []
        
        for read in reads:
            n = len(read)
            for i in range(n-k+1):
                kmer = read[i:i+k]
                kmers.append(kmer)
                      
        return kmers    
    
    # Konstruisanje De Brujinovog grafa od k-grama 
    def __init__(self, reads, k):
        kmers = self.get_kmers(reads, k)
        adjacency_list = {}
        
        for kmer in kmers:
            u = kmer[:-1]
            v = kmer[1:]
            
            if u not in adjacency_list:
                adjacency_list[u] = []
            if v not in adjacency_list:
                adjacency_list[v] = []
                
            adjacency_list[u].append(v)
            
        super().__init__(adjacency_list)

In [32]:
reads = ['TAATGCCATGGGATGTT']   
db_graph = DeBruijn(reads, 3)
print(db_graph)

{'TA': ['AA'], 'AA': ['AT'], 'AT': ['TG', 'TG', 'TG'], 'TG': ['GC', 'GG', 'GT'], 'GC': ['CC'], 'CC': ['CA'], 'CA': ['AT'], 'GG': ['GG', 'GA'], 'GA': ['AT'], 'GT': ['TT'], 'TT': []}


<img src="figures/de_bruijn_graph.png" width="400">

In [33]:
db_graph.close_to_cycle()
res = db_graph.all_eulerian_cycles()

for i in res:
    print(i)

('TA', 'AA', 'AT', 'TG', 'GG', 'GG', 'GA', 'AT', 'TG', 'GC', 'CC', 'CA', 'AT', 'TG', 'GT', 'TT', 'TA')
('TA', 'AA', 'AT', 'TG', 'GC', 'CC', 'CA', 'AT', 'TG', 'GG', 'GG', 'GA', 'AT', 'TG', 'GT', 'TT', 'TA')
